In [30]:
%run '00_database_connectivity_setup.ipynb'

,x,y
0,0.778415,0.134201
1,0.902056,0.791205
2,0.751707,0.972293
3,0.788108,0.152631
4,0.306569,0.419736


In [31]:
%%execsql
set search_path to yelp;


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import display

%matplotlib inline

# Limit rows disp# Limit rows displayed in notebook
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)
pd.set_option('display.precision', 6)

/Users/asong/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


## Notes on Data

__The Challenge Dataset:__

+ 2.2M reviews and 591K tips by 552K users for 77K businesses
+ 566K business attributes, e.g., hours, parking availability, ambience.
+ Social network of 552K users for a total of 3.5M social edges.
+ Aggregated check-ins over time for each of the 77K businesses
+ 200,000 pictures from the included businesses

__Cities:__

+ U.K.: Edinburgh
+ Germany: Karlsruhe
+ Canada: Montreal and Waterloo
+ U.S.: Pittsburgh, Charlotte, Urbana-Champaign, Phoenix, Las Vegas, Madison


### business
```json
{
    'type': 'business',
    'business_id': (encrypted business id),
    'name': (business name),
    'neighborhoods': [(hood names)],
    'full_address': (localized address),
    'city': (city),
    'state': (state),
    'latitude': latitude,
    'longitude': longitude,
    'stars': (star rating, rounded to half-stars),
    'review_count': review count,
    'categories': [(localized category names)]
    'open': True / False (corresponds to closed, not business hours),
    'hours': {
        (day_of_week): {
            'open': (HH:MM),
            'close': (HH:MM)
        },
        ...
    },
    'attributes': {
        (attribute_name): (attribute_value),
        ...
    },
}
```

### review
```json
{
    'type': 'review',
    'business_id': (encrypted business id),
    'user_id': (encrypted user id),
    'stars': (star rating, rounded to half-stars),
    'text': (review text),
    'date': (date, formatted like '2012-03-14'),
    'votes': {(vote type): (count)},
    'review_id': (encrypted review id)
}
```

### user
```json
{
    'type': 'user',
    'user_id': (encrypted user id),
    'name': (first name),
    'review_count': (review count),
    'average_stars': (floating point average, like 4.31),
    'votes': {(vote type): (count)},
    'friends': [(friend user_ids)],
    'elite': [(years_elite)],
    'yelping_since': (date, formatted like '2012-03'),
    'compliments': {
        (compliment_type): (num_compliments_of_this_type),
        ...
    },
    'fans': (num_fans),
}
```

### check-in
```json
{
    'type': 'checkin',
    'business_id': (encrypted business id),
    'checkin_info': {
        '0-0': (number of checkins from 00:00 to 01:00 on all Sundays),
        '1-0': (number of checkins from 01:00 to 02:00 on all Sundays),
        ...
        '14-4': (number of checkins from 14:00 to 15:00 on all Thursdays),
        ...
        '23-6': (number of checkins from 23:00 to 00:00 on all Saturdays)
    }, # if there was no checkin for a hour-day block it will not be in the dict
}
```

### tip
```json
{
    'type': 'tip',
    'text': (tip text),
    'business_id': (encrypted business id),
    'user_id': (encrypted user id),
    'date': (date, formatted like '2012-03-14'),
    'likes': (count),
}
```

## Business Data

In [194]:
%%showsql
select * from yelp.business_raw limit 10

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,"{""By Appointment Only"": false, ""Parking"": {""garage"": false, ""street"": false, ""validated"": false, ""lot"": false, ""valet"": false}, ""Wheelchair Accessible"": true}","""ll-z6OsHZnIYZqUJXC4Yug""","[""Hair Salons"", ""Beauty & Spas""]","""Gilbert""","""1652 N Higley Rd\nGilbert, AZ 85234""",{},33.3803157,-111.7224984,"""Vatoci Hair Studio""",[],true,4,4.5,"""AZ""","""business""\r"
1,"{""Accepts Credit Cards"": true, ""Price Range"": 2, ""Parking"": {""garage"": false, ""street"": false, ""validated"": false, ""lot"": true, ""valet"": false}}","""ZdXyTqHMier_fdiW7xbU4g""","[""Bakeries"", ""Grocery"", ""Shopping"", ""Florists"", ""Food"", ""Flowers & Gifts""]","""Mesa""","""9101 E Baseline Road\nMesa, AZ 85209""",{},33.37704,-111.638793,"""Safeway Food & Drug""",[],true,11,2.5,"""AZ""","""business""\r"
2,"{""Take-out"": true, ""Drive-Thru"": false, ""Outdoor Seating"": false, ""Caters"": false, ""Noise Level"": ""average"", ""Parking"": {""garage"": false, ""street"": false, ""validated"": false, ""lot"": true, ""valet"": false}, ""Good For Groups"": true, ""Delivery"": false, ""Attire"": ""casual"", ""Has TV"": true, ""Price Range"": 1, ""Good For"": {""dessert"": false, ""latenight"": false, ""lunch"": false, ""dinner"": false, ""brunch"": false, ""breakfast"": true}, ""Takes Reservations"": false, ""Ambience"": {""romantic"": false, ""intimate"": false, ""classy"": false, ""hipster"": false, ""divey"": false, ""touristy"": false, ""trendy"": false, ""upscale"": false, ""casual"": true}, ""Waiter Service"": true, ""Accepts Credit Cards"": true, ""Good for Kids"": true, ""Alcohol"": ""none"", ""Wheelchair Accessible"": true}","""sdnl07WmIzVxqNQv6DdyHw""","[""Burgers"", ""Breakfast & Brunch"", ""Salad"", ""Restaurants""]","""Chandler""","""2780 E Germann Rd\nChandler, AZ 85248""","{""Monday"": {""close"": ""00:00"", ""open"": ""05:30""}, ""Tuesday"": {""close"": ""00:00"", ""open"": ""05:30""}, ""Friday"": {""close"": ""01:00"", ""open"": ""05:30""}, ""Wednesday"": {""close"": ""00:00"", ""open"": ""05:30""}, ""Thursday"": {""close"": ""00:00"", ""open"": ""05:30""}, ""Sunday"": {""close"": ""00:00"", ""open"": ""05:30""}, ""Saturday"": {""close"": ""01:00"", ""open"": ""05:30""}}",33.2792396,-111.7928241,"""Village Inn""",[],true,21,3.5,"""AZ""","""business""\r"
3,"{""Price Range"": 3, ""Accepts Credit Cards"": true, ""Wheelchair Accessible"": true, ""Parking"": {""garage"": false, ""street"": false, ""validated"": false, ""lot"": true, ""valet"": false}}","""kq7esIwZpYXLKPJhHMIZgg""","[""Sporting Goods"", ""Shopping""]","""Chandler""","""Crossroads Towne Center\n2650 E Germann Rd\nChandler, AZ 85249""","{""Monday"": {""close"": ""21:30"", ""open"": ""09:30""}, ""Tuesday"": {""close"": ""21:30"", ""open"": ""09:30""}, ""Friday"": {""close"": ""21:30"", ""open"": ""09:30""}, ""Wednesday"": {""close"": ""21:30"", ""open"": ""09:30""}, ""Thursday"": {""close"": ""21:30"", ""open"": ""09:30""}, ""Sunday"": {""close"": ""19:00"", ""open"": ""10:00""}, ""Saturday"": {""close"": ""21:00"", ""open"": ""09:00""}}",33.2773591,-111.7917787,"""Sport Chalet""",[],true,5,4.0,"""AZ""","""business""\r"
4,"{""Accepts Credit Cards"": true}","""inDYSqSDMyrfBhOZc702rw""","[""Professional Services"", ""Television Service Providers"", ""Home Services"", ""Internet Service Providers""]","""Phoenix""","""Phoenix, AZ 85027""","{""Monday"": {""close"": ""00:00"", ""open"": ""00:00""}, ""Tuesday"": {""close"": ""00:00"", ""open"": ""00:00""}, ""Friday"": {""close"": ""00:00"", ""open"": ""00:00""}, ""Wednesday"": {""close"": ""00:00"", ""open"": ""00:00""}, ""Thursday"": {""close"": ""00:00"", ""open"": ""00:00""}, ""Sunday"": {""close"": ""00:00"", ""open"": ""00:00""}, ""Saturday"": {""close"": ""00:00"", ""open"": ""00:00""}}",33.6782379,-112.1012912,"""Cox Communications""",[],true,294,1.5,"""AZ""","""business

### Categories

In [28]:
%%execsql
create or replace function split_array(
    arr text
) returns setof text as
$$
    try:
        arr_split = arr[1:-1].replace('"','').split(',')
        if not arr_split[0]:
            return []
        else:
            return tuple(arr_split)
    except:
        return []
$$ language plpythonu;

create or replace function split_array(
    arr text[]
) returns setof text as
$$
    return tuple(arr)
$$ language plpythonu;

In [26]:
%%execsql
drop table if exists business_categories;
create table business_categories as
select
    trim(both '"' from business_id) as business_id
    ,trim(both ' ' from split_array(categories)) as category
from business_raw 
distributed by (business_id);

In [12]:
%%showsql
select * from business_categories limit 5;

,business_id,category
0,TLrX4FECQDFz-E45TI_9ow,Seafood
1,TLrX4FECQDFz-E45TI_9ow,Restaurants
2,WUtPI9rJbs7ET1gPhULnDw,Southern
3,WUtPI9rJbs7ET1gPhULnDw,Restaurants
4,iLwNx5TVKVFKyV5RdbIY-A,Sushi Bars


## Limit scope of data to restaurants in US and Canada

In [36]:
%%execsql
drop table if exists restaurants_us_ca;
create table restaurants_us_ca as
select 
    *
    ,case when state='AZ' then 'Phoenix'
    when state='PA' then 'Pittsburgh'
    when state='IL' then 'Urbana-Champaign'
    when state='NV' then 'Las Vegas'
    when state='WI' then 'Madison'
    when state='NC' then 'Charlotte'
    when state='ON' then 'Waterloo'
    when state='QC' then 'Montreal'
    else null end as city_1
from 
    business_raw
join (
    select 
        '"'||business_id||'"' as business_id 
    from 
        business_categories 
    where 
        category ilike ANY(array['restaurants', 'food']) 
    group by 1
) cats
using (business_id)
where 
    state ilike ANY(array['AZ','PA','IL','NV','WI', 'NC', 'ON','QC'])
distributed by (business_id);

In [43]:
%%showsql
select 
    *
    ,case when state='AZ' then 'Phoenix'
    when state='PA' then 'Pittsburgh'
    when state='IL' then 'Urbana-Champaign'
    when state='NV' then 'Las Vegas'
    when state='WI' then 'Madison'
    when state='NC' then 'Charlotte'
    when state='ON' then 'Waterloo'
    when state='QC' then 'Montreal'
    else null end as city_1
from 
    business_raw
join (
    select 
        '"'||business_id||'"' as business_id 
    from 
        business_categories 
    where 
        category ilike ANY(array['restaurants', 'food']) 
    group by 1
) cats
using (business_id) limit 5

,business_id,attributes,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,city_1
0,"""2PfavOTufsPCRdYm-bFcpw""","{""Price Range"": 1, ""Accepts Credit Cards"": false, ""Wheelchair Accessible"": true, ""Wi-Fi"": ""no"", ""Parking"": {""garage"": false, ""street"": false, ""validated"": false, ""lot"": true, ""valet"": false}}","[""Food"", ""Ice Cream & Frozen Yogurt""]","""Pittsburgh""","""4600 E Carson St\nSouth Side\nPittsburgh, PA 15210""","{""Monday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Tuesday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Friday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Wednesday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Thursday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Sunday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Saturday"": {""close"": ""22:00"", ""open"": ""10:00""}}",40.4115087,-79.9558863,"""Page Dairy Mart""","[""South Side""]",true,53,4.5,"""PA""","""business""\r",None
1,"""2VuPoaFPktvtAqTX_Fzc-A""","{""Take-out"": true, ""Accepts Credit Cards"": true, ""Price Range"": 1, ""Outdoor Seating"": false, ""Caters"": false, ""Noise Level"": ""average"", ""Parking"": {""garage"": false, ""street"": false, ""validated"": false, ""lot"": false, ""valet"": false}, ""Delivery"": false, ""Attire"": ""casual"", ""Has TV"": true, ""Good For"": {""dessert"": false, ""latenight"": false, ""lunch"": false, ""dinner"": false, ""brunch"": false, ""breakfast"": false}, ""Takes Reservations"": false, ""Ambience"": {""romantic"": false, ""intimate"": false, ""touristy"": false, ""hipster"": false, ""divey"": false, ""classy"": false, ""trendy"": false, ""upscale"": false, ""casual"": false}, ""Waiter Service"": true, ""Wi-Fi"": ""no"", ""Good for Kids"": true, ""Good For Groups"": true, ""Alcohol"": ""full_bar""}","[""Mexican"", ""Restaurants""]","""Mount Holly""","""791 W Charlotte Ave\nMount Holly, NC 28120""",{},35.3035634,-81.0272196,"""El Valle Mexican Restaurant""",[],true,8,4.0,"""NC""","""business""\r",None
2,"""lX3XYcMWoRlHjrmkoGNefQ""","{""Take-out"": true, ""Accepts Credit Cards"": true, ""Price Range"": 1, ""Outdoor Seating"": false, ""Caters"": true, ""Noise Level"": ""average"", ""Parking"": {""garage"": false, ""street"": false, ""validated"": false, ""lot"": true, ""valet"": false}, ""Good For Groups"": true, ""Delivery"": true, ""Dogs Allowed"": false, ""Attire"": ""casual"", ""Has TV"": true, ""Good For"": {""dessert"": false, ""latenight"": false, ""lunch"": true, ""dinner"": false, ""brunch"": false, ""breakfast"": false}, ""Takes Reservations"": false, ""Ambience"": {""romantic"": false, ""intimate"": false, ""classy"": false, ""hipster"": false, ""divey"": false, ""touristy"": false, ""trendy"": false, ""upscale"": false, ""casual"": true}, ""Waiter Service"": false, ""Wi-Fi"": ""free"", ""Good for Kids"": true, ""Alcohol"": ""none"", ""Wheelchair Accessible"": false}","[""Chicken Wings"", ""Restaurants""]","""Pineville""","""9101 Pineville Matthews Rd\nSte H\nPineville, NC 28134""","{""Monday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Tuesday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Friday"": {""close"": ""23:00"", ""open"": ""10:00""}, ""Wednesday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Thursday"": {""close"": ""22:00"", ""open"": ""10:00""}, ""Sunday"": {""close"": ""21:00"", ""open"": ""12:00""}, ""Saturday"": {""close"": ""23:00"", ""open"": ""11:00""}}",35.086948,-80.871878,"""TLC Wings & Grill""",[],true,12,4.0,"""NC""","""business""\r",None
3,"""bewtZL-VV1oeGUdAhHVz6Q""","{""Take-out"": false, ""Price Range"": 3, ""Outdoor Seating"": false, ""Noise Level"": ""quiet"", ""Parking"": {""garage"": false, ""street"": false, ""validated"": false, ""lot"": true, ""valet"": false}, ""Delivery"": false, ""Attire"": ""casual"", ""Has TV"": false, ""Good For"": {""dessert"": false, ""latenight"": false, ""lunch"": false, ""dinner"": false, ""breakfast"": false, ""brun

### attributes
Attributes are json objects. Parse these into a separate table

In [39]:
%%execsql
drop type if exists attributes cascade;
create type attributes as (
    key text
    ,value text
);

create or replace function flatten_json(
    json_str text
    ,prefix text
) returns setof attributes as 
$$
    import json
    
    def flatten_json(json_dict, data =[], key=''):    
        for i in json_dict.items():
            if type(i[1]) != dict:
                data.append([(key+'.'+i[0]).replace(' ','_'), i[1]])
            else:
                data.extend(flatten_json(i[1], [], key+'.'+i[0]))
        return data

    return tuple(flatten_json(json.loads(json_str), key=prefix))
    
$$ language plpythonu;

In [99]:
%%execsql
-- table of attributes unnested 
drop table if exists business_attributes_unnest;
create table business_attributes_unnest as
with attributes as ( -- table of attributes unnested and any duplicates removed
    select 
        trim(both '"' from business_id) as business_id
        ,lower((attr).key) as key
        ,lower((attr).value) as value
    from (
        select
            business_id
            ,flatten_json(attributes, 'attr') as attr
        from business_raw
    ) t1
    GROUP BY 1,2,3
)
select 
     business_id
    ,key
    ,value
from 
    attributes
where key !='attr.good_for_kids'
UNION 
select 
    business_id
    ,key
    ,array_to_string(array_agg(value),',') AS value
from 
    attributes
where key ='attr.good_for_kids'
group by 1,2
distributed by (key);

In [100]:
%%showsql
select * from business_attributes_unnest limit 5;

,business_id,key,value
0,1wIKHSHPOkywuOkyrXt6dw,attr.music.video,false
1,5BXNeSwzOy1nhU9PtCHvIw,attr.music.video,false
2,cLojUtqkFpVIHFEvA_5Atw,attr.music.video,false
3,fytNHpJ5Js9hkD_cQO8rBA,attr.music.video,false
4,gTm2TSYkC44iMnI_Nzpm4Q,attr.music.video,false


__Let's take a look at the different attributes available__

In [139]:
%%showsql
select
    key
    ,count(*)
from 
    business_attributes_unnest
group by 
    1
order by 1;

,key,count
0,attr.accepts_credit_cards,45067
1,attr.accepts_insurance,458
2,attr.ages_allowed,235
3,attr.alcohol,20457
4,attr.ambience.casual,18531
5,attr.ambience.classy,18531
6,attr.ambience.divey,17546
7,attr.ambience.hipster,18309
8,attr.ambience.intimate,18531
9,attr.ambience.romantic,18531


__good_for attributes__

Looks like there are two types of "good for" attributes: 

1) good for type of eating
    + attr.good_for.breakfast
    + attr.good_for.brunch
    + attr.good_for.dessert
    + attr.good_for.dinner
    + attr.good_for.latenight
    + attr.good_for.lunch
2) good for abience/activity
    + attr.good_for_dancing
    + attr.good_for_groups
    + attr.good_for_kids

__Pivot the attributes table__

In [6]:
%%execsql
-- convert booleans to binary (in text for now)
drop table if exists business_attributes_unnest_temp;
create temp table business_attributes_unnest_temp as
select
    business_id
    ,key
    ,case when value='false' then '0'
     when value='true' then '1'
    else value end as value
from 
    business_attributes_unnest
distributed by (business_id);


-- this table is going to have boolean values (0,1) as text
drop table if exists business_attributes_temp;
select sugarlib.sugar($$
    create temp table business_attributes_temp as
    select business_id
        ,sugarlib.pivot(key,business_attributes_unnest_temp,value)
    from 
        business_attributes_unnest_temp
    group by business_id
    distributed by (business_id);
$$);

In [12]:
%%showsql
select * from business_attributes_temp limit 5;

,business_id,key_attr.accepts_credit_cards,key_attr.accepts_insurance,key_attr.ages_allowed,key_attr.alcohol,key_attr.ambience.casual,key_attr.ambience.classy,key_attr.ambience.divey,key_attr.ambience.hipster,key_attr.ambience.intimate,key_attr.ambience.romantic,key_attr.ambience.touristy,key_attr.ambience.trendy,key_attr.ambience.upscale,key_attr.attire,key_attr.by_appointment_only,key_attr.byob,key_attr.byob/corkage,key_attr.caters,key_attr.coat_check,key_attr.corkage,key_attr.delivery,key_attr.dietary_restrictions.dairy-free,key_attr.dietary_restrictions.gluten-free,key_attr.dietary_restrictions.halal,key_attr.dietary_restrictions.kosher,key_attr.dietary_restrictions.soy-free,key_attr.dietary_restrictions.vegan,key_attr.dietary_restrictions.vegetarian,key_attr.dogs_allowed,key_attr.drive-thru,key_attr.good_for.breakfast,key_attr.good_for.brunch,key_attr.good_for_dancing,key_attr.good_for.dessert,key_attr.good_for.dinner,key_attr.good_for_groups,key_attr.good_for_kids,key_attr.good_for.latenight,key_attr.good_for.lunch,key_attr.hair_types_specialized_in.africanamerican,key_attr.hair_types_specialized_in.asian,key_attr.hair_types_specialized_in.coloring,key_attr.hair_types_specialized_in.curly,key_attr.hair_types_specialized_in.extensions,key_attr.hair_types_specialized_in.kids,key_attr.hair_types_specialized_in.perms,key_attr.hair_types_specialized_in.straightperms,key_attr.happy_hour,key_attr.has_tv,key_attr.music.background_music,key_attr.music.dj,key_attr.music.jukebox,key_attr.music.karaoke,key_attr.music.live,key_attr.music.playlist,key_attr.music.video,key_attr.noise_level,key_attr.open_24_hours,key_attr.order_at_counter,key_attr.outdoor_seating,key_attr.parking.garage,key_attr.parking.lot,key_attr.parking.street,key_attr.parking.valet,key_attr.parking.validated,key_attr.payment_types.amex,key_attr.payment_types.cash_only,key_attr.payment_types.discover,key_attr.payment_types.mastercard,key_attr.payment_types.visa,key_attr.price_range,key_attr.smoking,key_attr.take-out,key_attr.takes_reservations,key_attr.waiter_service,key_attr.wheelchair_accessible,key_attr.wi-fi
0,08xNB6lB-2kMrj5bqJaEMA,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,0,0,0,0,None,None,None,None,None,4,None,None,None,None,None,None
1,0Ayzd0haow5Ek3CdfZg5bA,1,None,None,beer_and_wine,0,0,0,0,0,0,1,0,0,casual,None,None,None,0,None,None,0,None,None,None,None,None,None,None,None,None,0,0,None,0,0,0,1,0,1,None,None,None,None,None,None,None,None,None,1,None,None,None,None,None,None,None,average,None,None,0,0,0,0,0,0,None,None,None,None,None,2,None,1,0,1,None,None
2,0btslkqA-YJQjLH6GFjU-g,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,0gABc2O0CsMqhT35LPHfNQ,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,0m-dEwpqVNO4mgZ1mM7Wew,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [13]:
%%showsql
-- binary boolean values that need to be casted to integer type
select key from business_attributes_unnest where value not in ('true', 'false') group by 1;

,key
0,attr.byob/corkage
1,attr.smoking
2,attr.wi-fi
3,attr.ages_allowed
4,attr.alcohol
5,attr.good_for_kids
6,attr.noise_level
7,attr.price_range
8,attr.attire


In [16]:
%%execsql
drop table if exists business_attributes;
create table business_attributes as 
select
    business_id
    ,"key_attr.payment_types.visa"::integer as "payment_types.visa"
    ,"key_attr.hair_types_specialized_in.kids"::integer as "hair_types_specialized_in.kids"
    ,"key_attr.dietary_restrictions.dairy-free"::integer as "dietary_restrictions.dairy-free"
    ,"key_attr.hair_types_specialized_in.perms"::integer as "hair_types_specialized_in.perms"
    ,"key_attr.dietary_restrictions.kosher"::integer as "dietary_restrictions.kosher"
    ,"key_attr.hair_types_specialized_in.straightperms"::integer as "hair_types_specialized_in.straightperms"
    ,"key_attr.music.karaoke"::integer as "music.karaoke"
    ,"key_attr.payment_types.discover"::integer as "payment_types.discover"
    ,"key_attr.order_at_counter"::integer as "order_at_counter"
    ,"key_attr.dietary_restrictions.halal"::integer as "dietary_restrictions.halal"
    ,"key_attr.dietary_restrictions.gluten-free"::integer as "dietary_restrictions.gluten-free"
    ,"key_attr.payment_types.amex"::integer as "payment_types.amex"
    ,"key_attr.happy_hour"::integer as "happy_hour"
    ,"key_attr.hair_types_specialized_in.extensions"::integer as "hair_types_specialized_in.extensions"
    ,"key_attr.corkage"::integer as "corkage"
    ,"key_attr.dietary_restrictions.vegetarian"::integer as "dietary_restrictions.vegetarian"
    ,"key_attr.hair_types_specialized_in.curly"::integer as "hair_types_specialized_in.curly"
    ,"key_attr.accepts_insurance"::integer as "accepts_insurance"
    ,"key_attr.music.playlist"::integer as "music.playlist"
    ,"key_attr.good_for_dancing"::integer as "good_for_dancing"
    ,"key_attr.byob"::integer as "byob"
    ,"key_attr.hair_types_specialized_in.asian"::integer as "hair_types_specialized_in.asian"
    ,"key_attr.music.video"::integer as "music.video"
    ,"key_attr.payment_types.cash_only"::integer as "payment_types.cash_only"
    ,"key_attr.hair_types_specialized_in.africanamerican"::integer as "hair_types_specialized_in.africanamerican"
    ,"key_attr.by_appointment_only"::integer as "by_appointment_only"
    ,"key_attr.caters"::integer as "caters"
    ,"key_attr.ambience.intimate"::integer as "ambience.intimate"
    ,"key_attr.ambience.upscale"::integer as "ambience.upscale"
    ,"key_attr.ambience.divey"::integer as "ambience.divey"
    ,"key_attr.waiter_service"::integer as "waiter_service"
    ,"key_attr.open_24_hours"::integer as "open_24_hours"
    ,"key_attr.ambience.trendy"::integer as "ambience.trendy"
    ,"key_attr.payment_types.mastercard"::integer as "payment_types.mastercard"
    ,"key_attr.good_for.breakfast"::integer as "good_for.breakfast"
    ,"key_attr.ambience.classy"::integer as "ambience.classy"
    ,"key_attr.ambience.romantic"::integer as "ambience.romantic"
    ,"key_attr.ambience.touristy"::integer as "ambience.touristy"
    ,"key_attr.good_for.lunch"::integer as "good_for.lunch"
    ,"key_attr.outdoor_seating"::integer as "outdoor_seating"
    ,"key_attr.wheelchair_accessible"::integer as "wheelchair_accessible"
    ,"key_attr.music.jukebox"::integer as "music.jukebox"
    ,"key_attr.music.background_music"::integer as "music.background_music"
    ,"key_attr.has_tv"::integer as "has_tv"
    ,"key_attr.take-out"::integer as "take-out"
    ,"key_attr.drive-thru"::integer as "drive-thru"
    ,"key_attr.good_for_groups"::integer as "good_for_groups"
    ,"key_attr.dietary_restrictions.soy-free"::integer as "dietary_restrictions.soy-free"
    ,"key_attr.takes_reservations"::integer as "takes_reservations"
    ,"key_attr.delivery"::integer as "delivery"
    ,"key_attr.coat_check"::integer as "coat_check"
    ,"key_attr.parking.valet"::integer as "parking.valet"
    ,"key_attr.parking.lot"::integer as "parking.lot"
    ,"key_attr.dietary_restrictions.vegan"::integer as "dietary_restrictions.vegan"
    ,"key_attr.good_for.brunch"::integer as "good_for.brunch"
    ,"key_attr.good_for.dinner"::integer as "good_for.dinner"
    ,"key_attr.hair_types_specialized_in.coloring"::integer as "hair_types_specialized_in.coloring"
    ,"key_attr.music.dj"::integer as "music.dj"
    ,"key_attr.good_for.dessert"::integer as "good_for.dessert"
    ,"key_attr.good_for.latenight"::integer as "good_for.latenight"
    ,"key_attr.parking.garage"::integer as "parking.garage"
    ,"key_attr.dogs_allowed"::integer as "dogs_allowed"
    ,"key_attr.parking.validated"::integer as "parking.validated"
    ,"key_attr.music.live"::integer as "music.live"
    ,"key_attr.parking.street"::integer as "parking.street"
    ,case when "key_attr.good_for_kids"='1' then 1 
    when "key_attr.good_for_kids"='0' then 0
    else 2 end as "good_for_kids" -- 2 means it was both true and false in the json ie ambiguous
    ,"key_attr.ambience.hipster"::integer as "ambience.hipster"
    ,"key_attr.accepts_credit_cards"::integer as "accepts_credit_cards"
    ,"key_attr.ambience.casual"::integer as "ambience.casual"
    ,"key_attr.byob/corkage" as "byob/corkage"
    ,"key_attr.smoking" as "smoking"
    ,"key_attr.wi-fi" as "wi-fi"
    ,"key_attr.ages_allowed" as "ages_allowed"
    ,"key_attr.alcohol" as "alcohol"
    ,"key_attr.noise_level" as "noise_level"
    ,"key_attr.price_range" as "price_range"
    ,"key_attr.attire" as "attire"
from 
    business_attributes_temp
distributed by (business_id);

In [200]:
%%execsql
drop type if exists count_type cascade;
create type count_type as (
    col text
    ,cnt integer
    ,prob double precision
    ,cume_prob double precision
);

create or replace function count_summary(
    col text
    ,table_name text
) returns setof count_type as
$$
    import ast
    sql = """
        with summary as (
            select 
                '"'||"{col}"||'"' as col
                ,count(*) as cnt
                ,count(*)/sum(count(*)) OVER () as prob
                ,sum(count(*)) over(order by count(*) desc rows unbounded preceding) / sum(count(*)) over () as cume_prob
            from 
                {table_name}
            group by 1
        )
        select summary from summary;
    """.format(col=col, table_name=table_name)
    res = plpy.execute(sql)
    
    results = []
    for rec in res:
        s = rec['summary'].replace('(,', '(None,')
        plpy.info(s)
        results.append(ast.literal_eval(s))
    return tuple(results)
$$ language plpythonu;

In [219]:
sql = """select column_name::text from information_schema.columns where table_name='business_attributes' and table_schema='yelp' order by ordinal_position"""

df = psql.read_sql(sql,conn)
conn.commit()
columns = df.column_name.tolist()[1:]
table_name = 'yelp.business_attributes'
fails = []

for col in columns:
    try:
        sql="""select * from count_summary('{col}', '{table_name}');""".format(table_name=table_name, col=col)
        df = psql.read_sql(sql,conn)
        conn.commit()
        display(col)
        display(df)
    except:
        fails.append(col)
        pass

'key_attr.accepts_credit_cards'

,col,cnt,prob,cume_prob
0,1,42889,0.814745,0.814745
1,None,7574,0.143880,0.958625
2,0,2178,0.041375,1.000000


'key_attr.accepts_insurance'

,col,cnt,prob,cume_prob
0,None,52183,0.991300,0.991300
1,0,391,0.007428,0.998727
2,1,67,0.001273,1.000000


'key_attr.ages_allowed'

...

,col,cnt,prob,cume_prob
0,None,33778,0.641667,0.641667
1,1,11908,0.226212,0.867879
2,0,6955,0.132121,1.000000


'key_attr.wheelchair_accessible'

,col,cnt,prob,cume_prob
0,None,33114,0.629053,0.629053
1,1,17843,0.338956,0.968010
2,0,1684,0.031990,1.000000


'key_attr.wi-fi'

,col,cnt,prob,cume_prob
0,None,34743,0.659999,0.659999
1,no,10182,0.193423,0.853422
2,free,7323,0.139112,0.992534
3,paid,393,0.007466,1.000000


### Notes on above
+ most businesses accept credit cards (81%)
+ insurance is not relevant to most businesses (99.1%)
+ 99.55% of businsesses don't have age specific restrictions
    - .3% of businesses have 21+ restriction

### hours

In [13]:
%%showsql
select business_id, hours from business_raw limit 5;

,business_id,hours
0,"""4eFIVmbeCvJGCrhzmd-VYA""","{""Monday"": {""close"": ""19:00"", ""open"": ""06:00""}, ""Tuesday"": {""close"": ""19:00"", ""open"": ""06:00""}, ""Friday"": {""close"": ""19:00"", ""open"": ""06:00""}, ""Wednesday"": {""close"": ""19:00"", ""open"": ""06:00""}, ""Thursday"": {""close"": ""19:00"", ""open"": ""06:00""}, ""Sunday"": {""close"": ""14:00"", ""open"": ""07:00""}, ""Saturday"": {""close"": ""16:00"", ""open"": ""07:00""}}"
1,"""EUXT1vSFWaiD_BFmFK8rBg""",{}
2,"""xfu2Ja977b0UA1jZZN72UA""","{""Monday"": {""close"": ""17:30"", ""open"": ""07:00""}, ""Tuesday"": {""close"": ""17:30"", ""open"": ""07:00""}, ""Friday"": {""close"": ""17:30"", ""open"": ""07:00""}, ""Wednesday"": {""close"": ""17:30"", ""open"": ""07:00""}, ""Thursday"": {""close"": ""17:30"", ""open"": ""07:00""}, ""Sunday"": {""close"": ""16:00"", ""open"": ""09:00""}, ""Saturday"": {""close"": ""17:00"", ""open"": ""07:00""}}"
3,"""nuGD-Ewf7sxts2b_z_DkwQ""","{""Monday"": {""close"": ""21:00"", ""open"": ""07:00""}, ""Tuesday"": {""close"": ""21:00"", ""open"": ""07:00""}, ""Friday"": {""close"": ""21:00"", ""open"": ""07:00""}, ""Wednesday"": {""close"": ""21:00"", ""open"": ""07:00""}, ""Thursday"": {""close"": ""21:00"", ""open"": ""07:00""}, ""Sunday"": {""close"": ""21:00"", ""open"": ""07:00""}, ""Saturday"": {""close"": ""21:00"", ""open"": ""07:00""}}"
4,"""eOPxvbzFTBjO5LSV3uYMug""",{}


In [14]:
%%showsql
select 
    trim(both '"' from business_id) as business_id
    ,(flatten_json(hours, 'hours')).* 
from business_raw  limit 5

,business_id,key,value
0,CHYRvicu85H-gnmrZY1-Zw,hours_Monday_close,21:00
1,CHYRvicu85H-gnmrZY1-Zw,hours_Monday_open,10:00
2,CHYRvicu85H-gnmrZY1-Zw,hours_Tuesday_close,21:00
3,CHYRvicu85H-gnmrZY1-Zw,hours_Tuesday_open,10:00
4,CHYRvicu85H-gnmrZY1-Zw,hours_Friday_close,21:00


In [84]:
%%execsql
-- function to clean the resulting parsed json values
drop type if exists hours cascade;
create type hours as (
    dayofweek text
    ,ops text
);

create or replace function hours_pprint(
    business_hours text
) returns hours as
$$
    hours = business_hours.split('.')
    return [hours[1], hours[2]] # [dayofweek, open/close]
$$ language plpythonu;

In [85]:
%%execsql
drop table if exists business_hours_temp;
create temp table business_hours_temp as
select
    business_id
    ,(hours_pprint(key)).*
    ,value as hour
from (
select 
    trim(both '"' from business_id) as business_id
    ,(flatten_json(hours, 'hours')).* 
from business_raw 
) t1
distributed by (business_id);

In [86]:
%%showsql
select * from business_hours_temp limit 5;

,business_id,dayofweek,ops,hour
0,TLrX4FECQDFz-E45TI_9ow,Monday,close,22:00
1,TLrX4FECQDFz-E45TI_9ow,Monday,open,17:30
2,TLrX4FECQDFz-E45TI_9ow,Tuesday,close,22:00
3,TLrX4FECQDFz-E45TI_9ow,Tuesday,open,17:30
4,TLrX4FECQDFz-E45TI_9ow,Friday,close,14:30


In [87]:
%%execsql
drop table if exists business_hours;
select sugarlib.sugar($$
    create table yelp.business_hours as
    select
        business_id
        ,dayofweek
        ,sugarlib.pivot(ops,business_hours_temp,hour)
    from 
        business_hours_temp
    group by 1,2
    distributed by (business_id);
$$);

In [88]:
%%showsql
select * from business_hours limit 5;

,business_id,dayofweek,ops_close,ops_open
0,05fTEsjUPVo5jks8PpNKCQ,Friday,01:30,10:00
1,05fTEsjUPVo5jks8PpNKCQ,Monday,20:00,10:00
2,05fTEsjUPVo5jks8PpNKCQ,Saturday,01:30,10:00
3,05fTEsjUPVo5jks8PpNKCQ,Sunday,22:00,08:00
4,05fTEsjUPVo5jks8PpNKCQ,Thursday,20:00,10:00


In [77]:
%%showsql
select 
    trim(both '"' from business_id) as business_id
    ,(flatten_json(hours, 'hours')).* 
from business_raw limit 5

,business_id,key,value
0,Y68Ysab60gJhX7YBjESgBg,hours.Tuesday.close,17:00
1,Y68Ysab60gJhX7YBjESgBg,hours.Tuesday.open,08:00
2,Y68Ysab60gJhX7YBjESgBg,hours.Friday.close,17:00
3,Y68Ysab60gJhX7YBjESgBg,hours.Friday.open,08:00
4,Y68Ysab60gJhX7YBjESgBg,hours.Monday.close,17:00


In [89]:
%%execsql
drop table if exists business_hours_temp;
create temp table business_hours_temp as
select 
    trim(both '"' from business_id) as business_id
    ,(flatten_json(hours, 'hours')).* 
from business_raw 
distributed by (business_id);

In [90]:
%%execsql
drop table if exists business_hours_wide;
select sugarlib.sugar($$
    create table yelp.business_hours_wide as
    select
        business_id
        ,sugarlib.pivot(key,business_hours_temp,value)
    from 
        business_hours_temp
    group by 1
    distributed by (business_id);
$$);

In [91]:
%%showsql
select * from business_hours_wide limit 5;

,business_id,key_hours.friday.close,key_hours.friday.open,key_hours.monday.close,key_hours.monday.open,key_hours.saturday.close,key_hours.saturday.open,key_hours.sunday.close,key_hours.sunday.open,key_hours.thursday.close,key_hours.thursday.open,key_hours.tuesday.close,key_hours.tuesday.open,key_hours.wednesday.close,key_hours.wednesday.open
0,08xNB6lB-2kMrj5bqJaEMA,00:00,10:00,00:00,10:00,00:00,10:00,00:00,10:00,00:00,10:00,00:00,10:00,00:00,10:00
1,0btslkqA-YJQjLH6GFjU-g,21:00,09:00,21:00,09:00,21:00,09:00,18:00,09:00,21:00,09:00,21:00,09:00,21:00,09:00
2,0gABc2O0CsMqhT35LPHfNQ,18:00,09:00,18:00,09:00,18:00,09:00,16:00,10:00,18:00,09:00,18:00,09:00,18:00,09:00
3,0m-dEwpqVNO4mgZ1mM7Wew,17:00,08:00,17:00,08:00,17:00,08:00,None,None,17:00,08:00,17:00,08:00,17:00,08:00
4,0NeXPGy3_N9T4IeHde7iGQ,22:00,11:00,21:00,11:00,22:00,11:00,20:00,11:00,21:00,11:00,21:00,11:00,21:00,11:00


### neighborhoods

In [52]:
%%showsql
select business_id, neighborhoods from business_raw where neighborhoods ilike '%,%' limit 5;

,business_id,neighborhoods
0,"""Pc932k0ZDQYxjt3RflTx8w""","[""Southeast"", ""University""]"
1,"""Dhz3Qj3RLOT1ublaAqBw9g""","[""Tollcross"", ""The Meadows""]"
2,"""TMhHqj5KnkxOBR9vdM84qg""","[""Cotswold"", ""Eastland""]"
3,"""rrD6XzX6fQ-aBnGczJIn-Q""","[""Old Town"", ""Royal Mile""]"
4,"""I0U4MQTiSNdGmpYoNcrimg""","[""West End"", ""New Town""]"


In [32]:
%%execsql
drop table if exists business_neighborhoods;
create table business_neighborhoods as
select trim(both '"' from business_id) as business_id, trim(both ' ' from split_array(neighborhoods)) as neighborhood
from business_raw 
distributed by (business_id);

In [24]:
%%showsql
select * from business_neighborhoods limit 5;

,business_id,neighborhood
0,0ilWo2kWlmP_Nv-P2rcNOA,Eken Park
1,nUISXnW2Jo9ovnoFBIUY_Q,Bloomfield
2,DmRw9flaIQ1VTdmjGspN6A,Westside
3,T4D7wX6GuCYChwRmser-0A,Brookline
4,NMqAOyNp6J3rH5B_RZBbbw,Leith


### Create meta data table for restaurants

In [34]:
%%execsql
drop table if exists businesses;
create table businesses as
with business as (
    select 
        trim(both '"' from business_id) as business_id
        ,trim(both '"' from city) as city
        ,trim(both '"' from full_address) as full_address
        ,latitude::double precision
        ,longitude::double precision
        ,trim(both '"' from name) as name
        ,cast_array(categories) as categories
        ,case when open = 'true' then 1 else 0 end as open
        ,review_count::integer
        ,stars::double precision
        ,cast_array(neighborhoods) as neighborhoods
        ,trim(both '"' from state) as state
    from 
        business_raw as business
)
select
    *
from 
    business 
left join
    business_hours_wide hours
using (business_id)
left join 
    business_attributes attr
using (business_id)
distributed by (business_id);

In [35]:
%%showsql
select * from businesses limit 5;

,business_id,city,full_address,latitude,longitude,name,categories,open,review_count,stars,neighborhoods,state,key_hours.friday.close,key_hours.friday.open,key_hours.monday.close,key_hours.monday.open,key_hours.saturday.close,key_hours.saturday.open,key_hours.sunday.close,key_hours.sunday.open,key_hours.thursday.close,key_hours.thursday.open,key_hours.tuesday.close,key_hours.tuesday.open,key_hours.wednesday.close,key_hours.wednesday.open,payment_types.visa,hair_types_specialized_in.kids,dietary_restrictions.dairy-free,hair_types_specialized_in.perms,dietary_restrictions.kosher,hair_types_specialized_in.straightperms,music.karaoke,payment_types.discover,order_at_counter,dietary_restrictions.halal,dietary_restrictions.gluten-free,payment_types.amex,happy_hour,hair_types_specialized_in.extensions,corkage,dietary_restrictions.vegetarian,hair_types_specialized_in.curly,accepts_insurance,music.playlist,good_for_dancing,byob,hair_types_specialized_in.asian,music.video,payment_types.cash_only,...,ambience.intimate,ambience.upscale,ambience.divey,waiter_service,open_24_hours,ambience.trendy,payment_types.mastercard,good_for.breakfast,ambience.classy,ambience.romantic,ambience.touristy,good_for.lunch,outdoor_seating,wheelchair_accessible,music.jukebox,music.background_music,has_tv,take-out,drive-thru,good_for_groups,dietary_restrictions.soy-free,takes_reservations,delivery,coat_check,parking.valet,parking.lot,dietary_restrictions.vegan,good_for.brunch,good_for.dinner,hair_types_specialized_in.coloring,music.dj,good_for.dessert,good_for.latenight,parking.garage,dogs_allowed,parking.validated,music.live,parking.street,good_for_kids,ambience.hipster,accepts_credit_cards,ambience.casual,byob/corkage,smoking,wi-fi,ages_allowed,alcohol,noise_level,price_range,attire
0,P7wKlbSRsoRrfH3DFOY08A,Phoenix,"2508 N Central Ave\nPhoenix, AZ 85004",33.475606,-112.074002,International House of Pancakes,"[Breakfast & Brunch, Restaurants]",1,43,3.0,None,AZ,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,0.0,0.0,0.0,1.0,None,0.0,None,1.0,0.0,0.0,0.0,0.0,0.0,1.0,None,None,0.0,1.0,None,1.0,None,0.0,0.0,None,0.0,1.0,None,0.0,0.0,None,None,0.0,0.0,0.0,None,0.0,None,0.0,1.0,0.0,1.0,1.0,None,None,no,None,none,average,1,casual
1,1MApscjpyygvanCdaesG3g,Charlotte,"4718 Sharon Rd.\nSharon Corners S/C\nSouth Park\nCharlotte, NC 28210",35.147245,-80.832369,Men's Wearhouse,"[Men's Clothing, Fashion, Shopping, Formal Wear]",1,6,3.0,[South Park],NC,21:00,10:00,21:00,10:00,21:00,09:30,18:00,11:00,21:00,10:00,21:00,10:00,21:00,10:00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,None,NaN,NaN,None,0.0,0.0,None,NaN,NaN,None,None,NaN,NaN,0.0,None,0.0,None,0.0,2.0,NaN,1.0,NaN,None,None,None,None,None,None,2,None
2,YHDKMl5ij09wncT3eTX0yg,Las Vegas,"3113 N Rainbow Blvd\nLas Vegas, NV 89108",36.216485,-115.242364,Famous Nails,"[Beauty & Spas, Nail Salons]",1,7,2.0,None,NV,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,None,NaN,None,NaN,NaN,None,0.0,0.0,None,NaN,NaN,None,None,NaN,NaN,0.0,None,0.0,None,0.0,2.0,NaN,1.0,NaN,None,None,None,None,None,None,1,None
3,tL8QOD03ibKRf5jeEuMTQQ,Chandler,"1000 N Arizona Ave\nChandler, AZ 85225",33.320904,-111.842059,Pizza Hut,"[Italian, Pizza, Chicken Wings, Restaurants]",1,11,2.0,None,AZ,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,0.0,0.0,0.0,1.0,None,0.0,None,0.0,0.0,0.0,0.0,1

## Review Data

In [4]:
%%showsql
select * from yelp.review_raw limit 5;

,business_id,review_date,review_id,stars,review_text,type,user_id,votes
0,"""LHyGQnuWR2q44bsrw1ZLAg""","""2009-03-10""","""0vOc47f2dyOdRdJlhuQ-wA""",4,"""Lots of options. The sandwiches are moderately priced and good. Avoid the cupcakes - they are dry.""","""review""","""l-MUNNvM1UKB-xXrfKI8rQ""","{""funny"": 0, ""useful"": 0, ""cool"": 0}"
1,"""LHyGQnuWR2q44bsrw1ZLAg""","""2010-02-25""","""loWTkwyzwf-yvr6-6-w7vQ""",3,"""Alright, bad news first. Sadly, the Bagel Factory wasn't quite as good as La Bagel Delight (my favorite Brooklyn bagel shop/deli). I went in hoping for something that would rival LBD's Mr. Bagel, and I'm sorry to say that in that regard, the Bagel Factory fell a bit short.\n\nOn the other hand, this is Pittsburgh, not New York City, and the Bagel Factory is by far the best bagel I've had here. Brueggers and Einstein don't even compare.\n\nThe egg in my bacon, egg and cheese bagel sandwich left a lot to be desired. It definitely seemed like a premade egg product that was simply heated for my sandwich. Fresh, scrambled eggs would have earned the Bagel Factory 4 stars here. I imagine this place would be excellent for the standard bagel and cream cheese.\n\nThe service was friendly and fast, the place was clean.""","""review""","""FuSWB3HKljZn6cUOzf6EMw""","{""funny"": 0, ""useful"": 1, ""cool"": 0}"
2,"""LHyGQnuWR2q44bsrw1ZLAg""","""2010-03-19""","""9JnE3t2o3ILxMxgAfijnxg""",2,"""It is rather unfortunate that Pittsburgh doesn't have ANY privately owned strictly bagel joints. Bagels, in my opinion, are one of the best breakfast alternatives to actually sitting down to a nice breakfast. I want this - being a student, basically full-time server, volunteering - my life pretty much requires a bagel breakfast every now and then. However, I'm in Pittsburgh, where can I go?\nEisteins? Brueggers? These place are virtually the same. Panera? I used to work here so not only do I dislike the taste, but I have a natural aversion to eating anything from this place. \nBagel Factory! Yes! No...these bagels are about the same you would get anywhere. Just as chewy, that taste of \""this is feigning fresh because it is warm and was just heated up, but it's definitely not fresh fresh.\""\nSadly, this is what Pittsburghers must settle for. I we can hope for is that one day a kind Philadelphian or New Yorker will show us the light, and bring us light, fluffy bagels of my dreams!""","""review""","""c7pBLYgkD3HOEY1IFXvf2w""","{""funny"": 2, ""useful"": 3, ""cool"": 1}"
3,"""LHyGQnuWR2q44bsrw1ZLAg""","""2011-03-20""","""iII3bV909_MlfFIUYkDudQ""",3,"""First thing, service is extremely good. Folks behind the counter and register were very helpful.\n\nThe bagels a smaller and more dense than what I am familiar with, so they could be more authentic, ala NY variety. Personally I am accustomed to and prefer the Noah's, Einstein's (OK I hate their use of corn meal as nonstick at stores but not Costco), Bruegger's bigger airier variety (Costco being one of the best places to procure these in bulk Noah's or Einstein's depending on geography, Einstein's in Pittsburgh).\n\nI had the daily sandwich which was chicken salad on a croissant ironically. This was very good. Unlimited self-service coffee refills (at least no one asked me to pay) was good too with two regular varieties and one decaf.\n\nAll in all I suspect opinion on the food will be what you think a bagel should be like...\n\nMetered street parking...""","""review""","""63Pop3d15pTuHZ7qoex46w""","{""funny"": 0, ""useful"": 0, ""cool"": 0}"
4,"""LHyGQnuWR2q44bsrw1ZLAg""","""2011-06-24""","""Yj6zM-3fUQgKRRWWUMCKcA""",4,"""Best bagels in Pittsburgh. It reminds me of the bagel shops I grew up with on Long Island. \n\nThe bagels are appropriately chewy, and they've got a great selection. I've had a bunch of different types (salt, everything, sesame) and they were all delicious. They also have a selection of desserts and other foods.""","""review""","""IJY44ja_rSkcfbMWExyboQ""","{""funny"": 0, "

### votes

In [5]:
%%execsql
drop table if exists review_votes_temp;
create temp table review_votes_temp as
select 
review_id 
,business_id 
,(flatten_json(votes, 'votes')).* 
from review_raw 
distributed by (review_id);

In [6]:
%%showsql
select * from review_votes_temp limit 5

,review_id,business_id,key,value
0,"""ISphyYy9XO_5-CY3M228GA""","""e5J3s-xxs1S6PPHJhldBEA""",votes_funny,0
1,"""ISphyYy9XO_5-CY3M228GA""","""e5J3s-xxs1S6PPHJhldBEA""",votes_useful,0
2,"""ISphyYy9XO_5-CY3M228GA""","""e5J3s-xxs1S6PPHJhldBEA""",votes_cool,0
3,"""_uJeFDqNqcgzUeKLNDJliw""","""Og03_vbBCVAFNHXgdflRLw""",votes_funny,0
4,"""_uJeFDqNqcgzUeKLNDJliw""","""Og03_vbBCVAFNHXgdflRLw""",votes_useful,0


In [7]:
%%execsql
drop table if exists review_votes;
select sugarlib.sugar($$
    create table review_votes as
    select review_id, business_id
        ,sugarlib.pivot(key,review_votes_temp,value)
    from 
        review_votes_temp
    group by review_id, business_id
    distributed by (review_id);
$$);

In [9]:
%%showsql
select * from review_votes limit 5

,review_id,business_id,key_votes_cool,key_votes_funny,key_votes_useful
0,"""0035PeaN1SlPq4JiEGU_dg""","""l2lpBXx8jKl87J2szyJRuQ""",1,0,1
1,"""00DkGNkWv6uTieRIVJ-opg""","""OedV_QX8aD7MeLqshhWkCQ""",0,0,0
2,"""00fhWT2o1qP3lUN1nxbm0w""","""QkAv4k5aB6SwNRNK7q19ug""",0,0,0
3,"""00tBGNrc_pQHFaJx4vdL_w""","""8KSK12p77qHb4gvN6DAncg""",1,0,2
4,"""01B60AUs02ckmliQqRBjPg""","""eFIwRnbZpToIwWOYWWJ7nA""",0,0,0


### Cleaned review data

In [4]:
%%execsql
drop table if exists review;
create table review as
select 
    trim(both '"' from t1.business_id) as business_id
    ,trim(both '"' from t1.review_id) as review_id 
    ,trim(both '"' from review_date)::date as review_date
    ,stars::integer
    ,trim(both '"' from review_text) as review_text 
    ,trim(both '"' from user_id) as user_id
    ,key_votes_cool::int as votes_cool
    ,key_votes_funny::int as votes_funny
    ,key_votes_useful::int as votes_useful 
from 
    review_raw t1
left join
    review_votes t2
using (review_id, business_id)
distributed by (review_id);

In [53]:
%%showsql
select * from review limit 3;

,business_id,review_id,review_date,stars,review_text,user_id,votes_cool,votes_funny,votes_useful
0,EEuGTah-oEssSl3o6pK-ZQ,wVLvNJHpmlrZ-m8jnh-cGQ,2008-10-07,3,"The pros: Dinosaurs in Their Time is pretty cool. The years of renovation were worth it - the former dinosaur hall is completely transformed. Remember how everything was in straight lines and the dinosaurs were up on their big pedestals? Now the exhibit is sort of S-shaped, the walls are painted with colorful murals, the support columns are disguised as trees, there are plants around the dinosaurs and dead leaves underneath. Underneath every dinosaur is a nameplate with a lot of information and a touchscreen with even MORE information, including a section on how the Carnegie Museum researchers contributed to the discovery or naming of this dinosaur. There is a lot more natural light than I remember, and there are 2 balconies that overlook the exhibits for more photo opportunities. There is also an extensive timeline of the Carnegie Museum's expeditions and contributions to paleontology, and a timelapse video of the renovations.\n\nThe cons: The rest of the history museum is pretty lame compared to this new and awesome exhibit. The biggest examples are the 2nd floor Halls of African and North American Wildlife and the 3rd floor Birds hall. Big cases of taxidermy in a wood-paneled room with dim lighting - hardly thrilling. The 3rd floor Polar World is starting to look dated too. Even the fonts used date the exhibits.\n\nAdmission is now $15 for adults, $11 for students (free for Pitt students!). This does get you into the Carnegie Museum of Art also, but I'm usually tired after doing the whole history museum so I never get over to the art side. I would recommend seeing the dinosaurs, then skipping the rest of the history museum if you've been there before. The Dinosaurs in Their Time would get five stars on its own, but the rest of the museum is lagging behind unfortunately.",wNtyBJa_qaRYUew_TWplgw,0,0,0
1,9NL2GL4jSPVj9SKijH5PqA,l9CVLRgt5uzQdakPQMffOg,2014-11-18,4,I haven't been to Kuhn's in months and forgot how much I prefer it over the other grocery stores in the area. Once you get past the craziness in the parking lot it's not so bad. It's smaller than the other grocery stores around but still have ample choices of all your produce/meat/dairy/deli needs fo sho. Their prices are less expensive than GE too! If only they had fuel perks... The service is always fast and friendly. I'll be doing the rest of my thanksgiving shopping there. They have some good deals going on right now. .99lb Honeysuckle turkeys no min purchase!,iyNf5jr-34Bg6nKOGEZXKg,0,0,0
2,105yyfGnNxJPPFbdIgBHLw,oUo17eaxek-rVwtnNP20LQ,2007-08-20,5,"TIS was definitely my favorite bookstore! It always had all my books, with used and new options. Reserving books online was definitely a time saver. It has a year round t-shirt deal, 3 for $20...hello Illinois t-shirt obsession, I now have over 20 Illinois t-shirts (I'm really serious). Sweatshirts are a little pricey here, but they tend to cost about the same at the other campus bookstores too. During the book purchasing rush, the store has plenty of cashiers, so the wait is never too long. Writing this review is making me miss TIS and college....",KUlS1BLQN05G4gEE7BlYdQ,0,0,0


## User Data

In [5]:
%%showsql
select * from yelp.user_raw limit 5;

average_stars  \
0  4.37           
1  3.11           
2  3.0            
3  3.49           
4  4.17           

                                                                                                                                compliments  \
0  {"funny": 4, "plain": 6, "writer": 1, "note": 1, "hot": 1, "cool": 5}                                                                      
1  {"writer": 1}                                                                                                                              
2  {}                                                                                                                                         
3  {"profile": 8, "funny": 22, "cute": 9, "plain": 70, "writer": 41, "list": 4, "note": 27, "photos": 50, "hot": 86, "cool": 90, "more": 8}   
4  {}                                                                                                                                         

          elite fans  \
0  []            1     
1  []            0     
2  []            0     
3  [2009, 2010]  32    
4  []            0     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

### compliments

In [62]:
%%execsql
drop table if exists user_compliments_temp;
create temp table user_compliments_temp as
select 
    trim(both '"' from user_id) as user_id
    ,(flatten_json(compliments, 'compliments')).*
from 
    user_raw
distributed by (user_id);

In [63]:
%%showsql
select * from user_compliments_temp limit 5;

,user_id,key,value
0,09XpyD1prjqKq4nS3XkKpA,compliments.plain,2
1,iEoPIDFBxiEuCVG8GzyENg,compliments.funny,4
2,iEoPIDFBxiEuCVG8GzyENg,compliments.plain,1
3,iEoPIDFBxiEuCVG8GzyENg,compliments.writer,3
4,iEoPIDFBxiEuCVG8GzyENg,compliments.note,4


In [16]:
%%showsql
select key, count(*) from user_compliments_temp group by 1 order by 2 desc;

,key,count
0,compliments_plain,84007
1,compliments_cool,77966
2,compliments_note,73508
3,compliments_writer,59562
4,compliments_hot,55896
...,...,...
6,compliments_more,44919
7,compliments_photos,29017
8,compliments_cute,22080
9,compliments_profile,19857


In [64]:
%%execsql
drop table if exists user_compliments;
select sugarlib.sugar($$
    create table yelp.user_compliments as
    select user_id as user_id
        ,sugarlib.pivot(key,user_compliments_temp,value)
    from 
        user_compliments_temp
    group by user_id
    distributed by (user_id);
$$);

In [65]:
%%showsql
select * from user_compliments limit 5;

,user_id,key_compliments.cool,key_compliments.cute,key_compliments.funny,key_compliments.hot,key_compliments.list,key_compliments.more,key_compliments.note,key_compliments.photos,key_compliments.plain,key_compliments.profile,key_compliments.writer
0,01nTwhHBaw8AHjQDb7JGBw,None,None,None,1,None,None,None,None,3,None,None
1,05qgjGUgH9Z8r1XRDRpkww,1,None,None,None,None,None,None,None,None,None,None
2,06YTa8A3QhWXv4l6Htr91A,4,None,5,None,None,1,9,None,5,None,1
3,07Ej2jE4VRoT0EmGtz8P5Q,None,None,1,None,None,None,None,None,None,None,None
4,07-MOraFwL_8CRJuzVYagA,1,None,1,None,None,1,1,None,None,None,1


### elite

In [70]:
%%showsql
select * from user_raw limit 5;

,average_stars,compliments,elite,fans,friends,name,review_count,type,user_id,votes,yelping_since
0,4.0,"{""note"": 1}",[],0,"[""4McyrxqtF0U61vijemLWHg"", ""4eM4nGh58BsLlm5lUXvuiQ""]","""Jeremy""",1,"""user""","""IhLF7__hHsRqL0SoPNviVA""","{""funny"": 0, ""useful"": 0, ""cool"": 0}","""2010-07"""
1,3.54,"{""note"": 1, ""plain"": 5, ""writer"": 1, ""cool"": 1}",[],4,"[""7b134wSLX8gN7G2sbLPZoQ"", ""hc4D2WkjKG8mk1C9e0fsQQ"", ""UoaC8YC4Z_rssccxV8nI-g"", ""F7VViCY5ja6RVykGb2P6qA"", ""CWzxV9y-UtHdnJASkFstzA""]","""Ronnie""",25,"""user""","""E4Nzd5GF6SREIOOcvhgCnQ""","{""funny"": 26, ""useful"": 62, ""cool"": 6}","""2010-09"""
2,3.36,{},[],0,[],"""Denny""",14,"""user""","""X6vAq_R9Ti7Wn-KGG0iWdg""","{""funny"": 0, ""useful"": 8, ""cool"": 0}","""2014-07"""
3,5.0,{},[],0,[],"""Amanda""",3,"""user""","""jRyiJ0gN1BAsoEUfxpvq5A""","{""funny"": 1, ""useful"": 2, ""cool"": 1}","""2013-04"""
4,4.23,"{""funny"": 14, ""plain"": 18, ""writer"": 29, ""note"": 6, ""photos"": 3, ""hot"": 63, ""more"": 4, ""cool"": 52}",[],5,"[""KtNT8biTJlbAFh-APgO7-w"", ""BroRJjahfi7v96ex-EMmPQ"", ""EhcZKrQJIS226xdm_qJ71w"", ""WmAyExqSWoiYZ5XEqpk_Uw"", ""-ANkfLbDf8aiBQ7vywIL6w"", ""0XRFyY8-dxmrDheFPpe8Sg"", ""pIm17SVRo8IdhGoBD4qbnA"", ""1aN5gfWMNfLkFMkNr60P-A"", ""ucKJFC8_cVK0NbW5uk4xTQ"", ""bCLFdVfkAiZ7t1xDskb66Q"", ""Iu3Jo9ROp2IWC9FwtWOaUQ"", ""eD9VAnbyW3a1Vg3bfLoX9w"", ""D6PxfJsBrrbAUSY3Rxf_CA"", ""oLAbWZXHwqOdbnVo9_mfww"", ""huKYBfeFtYCOWs4I-GESSw"", ""4p-qAdc_ZLXfieNwaZGNGA"", ""tWuvRDddJEZkayiL1shDUw"", ""uDVrgAX94cg2qndTSxsRqA"", ""xQpO4Jfi-Np3HLmJrQAe0Q"", ""9FDMUyJStI86B07bKQleow"", ""OW0TEeI9hGqsBv_nkawiNA"", ""Te7fSSZniESTOnaSCa3n8g"", ""rzHZ3iYVQe_8h2e42DbUsw"", ""vyiHjaQlESGcXgEfEmH4ow"", ""C0UY1f2BUUmLdmB3sn5_lw"", ""1IqWfrko5KycY9laJ30Gtw"", ""gqkKPS92L_qG03gHaEtIsQ"", ""jNwutFukLvxCjSFM-gJakQ"", ""kSVYpNWA19wUplbdi0U0Uw"", ""3DJLAIE0soSxYdvF50L5jQ"", ""9gWfA0MHsMoFedeizZre_g"", ""hcnHQ3NSgtASSAJHrD4ZxQ"", ""cs37Ohgq7_g3grJNEEUi2Q"", ""0RAmuyzHyrxhMSwAX-Hycg"", ""oOayyTrVO-PbVeXXsh_JEQ"", ""T3ISRzQ1fQPkUANyokIgAw"", ""76XTqNQr-DyqD9fZL-dMDg"", ""qBl3BXtZvA2Z_h8gGb3S7Q"", ""WZwfmfP5X1gDDeL-M9vzZA"", ""OaFcpi3W4AwxrD8W2pgC_A"", ""4RstSkgnodO4xNsktAiugA"", ""O40WWtg7RvUhY8GubqNkbA"", ""PRcfYRZGxTPFQgFqpnvp0Q"", ""_ZDMl1p5SZxWo1YETqk1PA"", ""Acp3GHrHX_bwILoZKNKjAA"", ""gBoMkFuGtVBUpJqtRL3qAw"", ""liuZC4JDxLampjR0Mi2G2A"", ""Yw-Q_4QrwWffjnHWLvo4kw"", ""ak3TEUwMIzNWu68IYQ8s3Q"", ""t5r4uvEflDd8rmi3GPU0Xg"", ""kufWkq1MqVW3CAqyS31Xpg"", ""zfb_dSwWV5mV4f_ZAgkYbg"", ""R_Fzh9YjpT0pg7dxhmFQnQ"", ""kUxLVldNw1z5NgF8Dth1Vw"", ""sDEZyS47IJ7ly9AVT7pXRQ"", ""3zNj0Sv5Cx-pnhX3LdjxAw"", ""W7okhz0hsKW7U7OGxFxnIw""]","""Lore""",52,"""user""","""JxgJymGG6H8Z_sCXlyg25A""","{""funny"": 150, ""useful"": 231, ""cool"": 188}","""2013-06"""


In [33]:
%%execsql
create or replace function cast_array(
    arr text
) returns text[] as 
$$
    import ast
    result = ast.literal_eval(arr)
    if result:
        return [i.strip() for i in result]
    else:
        return None
$$ language plpythonu;

In [23]:
%%showsql
select cast_array(elite) as elite from user_raw limit 5;

,elite
0,None
1,None
2,None
3,None
4,None


In [296]:
%%execsql
drop table if exists user_elite;
create table user_elite as
select 
    trim(both '"' from user_id) as user_id
    ,split_array(elite) as elite
from 
    user_raw
distributed by (user_id);

In [69]:
%%showsql
select * from user_elite limit 5

,user_id,elite
0,CdrOG7bBwAgHfkskPlA8xA,2011
1,CdrOG7bBwAgHfkskPlA8xA,2012
2,CdrOG7bBwAgHfkskPlA8xA,2013
3,CdrOG7bBwAgHfkskPlA8xA,2014
4,CdrOG7bBwAgHfkskPlA8xA,2015


### friends

In [299]:
%%execsql
create table user_friends as
select 
    trim(both '"' from user_id) as user_id
    ,split_array(friends) as friends
from 
    user_raw
distributed by (user_id);

In [300]:
%%showsql
select * from user_friends limit 5;

,user_id,friends
0,MjIlb4iQMEpEEDGNKoo97w,LGwoVXv6RwB7tEjgyeOZWw
1,qKfceRHpoYNIDV1HryX_hw,vWnUf5VXx9S7cbgomxsOww
2,GJ45sYsFmWuou9TZoTrJJg,VzAHWXVOIqylfCtIRXwU9w
3,GJ45sYsFmWuou9TZoTrJJg,7x3-B0yaO_hqXm42VKStzw
4,GJ45sYsFmWuou9TZoTrJJg,1ku6epbYu8IN2hmleuHvQg


### votes

In [56]:
%%execsql
drop table if exists user_votes_temp;
create temp table user_votes_temp as 
select
    user_id
    ,(votes).key as key
    ,(votes).value::integer as value
from (
    select
        trim(both '"' from user_id) as user_id
        ,(flatten_json(votes, 'votes')) as votes
    from 
        user_raw
) t1
distributed by (user_id);

In [57]:
%%showsql
select * from user_votes_temp limit 5;

,user_id,key,value
0,atvdPb748A_vq1uU6e6z4Q,votes.funny,19
1,atvdPb748A_vq1uU6e6z4Q,votes.useful,58
2,atvdPb748A_vq1uU6e6z4Q,votes.cool,23
3,edRbqZjCFLbOdMGDtnl5dQ,votes.funny,1
4,edRbqZjCFLbOdMGDtnl5dQ,votes.useful,6


In [58]:
%%execsql
drop table if exists user_votes;
select sugarlib.sugar($$
    create table yelp.user_votes as
    select user_id as user_id
        ,sugarlib.pivot(key,user_votes_temp,value)
    from 
        user_votes_temp
    group by user_id
    distributed by (user_id);
$$);

In [59]:
%%showsql
select * from user_votes limit 5;

,user_id,key_votes.cool,key_votes.funny,key_votes.useful
0,01jockG1cl78WWfcripIiQ,1,1,1
1,02-khcM_JJOHGU3Dk5EQ5A,0,0,2
2,04CG8ITB9L-7p1GNUxOoew,0,0,2
3,04Vn_yTfjddj3pBW6yF4qA,10,7,12
4,04WAMdJ1ddLLO1fG-EnA4A,14,9,44


In [73]:
%%execsql
drop table if exists users;
create table users as 
with userdata as (
select 
    average_stars::double precision
    ,fans::integer
    ,trim(both '"' from name) as name 
    ,review_count::integer 
    ,trim(both '"' from yelping_since) as yelping_since
    ,trim(both '"' from user_id) as user_id
    ,cast_array(elite) as elite
    ,cast_array(friends) as friends
from 
    user_raw 
)
select 
    *
from 
    userdata
left join 
    user_compliments usercomps
using (user_id)
left join user_votes uservotes
using (user_id)
distributed by (user_id);

In [121]:
%%showsql
select * from users limit 5

,user_id,average_stars,fans,name,review_count,yelping_since,elite,friends,key_compliments.cool,key_compliments.cute,key_compliments.funny,key_compliments.hot,key_compliments.list,key_compliments.more,key_compliments.note,key_compliments.photos,key_compliments.plain,key_compliments.profile,key_compliments.writer,key_votes.cool,key_votes.funny,key_votes.useful
0,gywsvSBCL6yCOmuXhluuBw,4.31,0,C,17,2007-02,[],"[pmPV9jgDyad6DK-jxZQ0PA, dw1zh9oNtZSSdhn9lrVdvA]",1,None,None,1,None,None,None,None,None,None,None,8,2,42
1,LyyualA8h35oLeOUxJxFbg,3.40,1,Sarah,65,2009-02,[],[],None,None,None,None,None,1,None,None,None,None,None,9,8,17
2,2h_EEqVKokJ1yqU8WAtXhA,3.60,0,Twinkle,20,2009-01,[],[],None,None,None,None,None,None,None,None,None,None,None,7,3,7
3,YKyJitb6wgvX2GAIyOcJ3g,4.00,0,Alex,1,2012-11,[],[],None,None,None,None,None,None,None,None,None,None,None,0,0,0
4,Ip123mRss7vVTngqearVXA,3.50,0,Nikita,53,2011-07,[],"[OqCgbvz3ZVsXzQreFvIWmw, clCE4R0bAjwp_L4IX3BvUg, meq4JCIvdj-zYbRS0s2ODQ]",None,None,None,None,None,None,None,None,None,None,None,3,4,22


## Check In Data

In [6]:
%%showsql
select * from yelp.checkin_raw limit 5;

,business_id,checkin_info,type
0,"""ra0evfke9lyf5sWzvKUbWQ""","{""10-4"": 1, ""15-5"": 1, ""10-2"": 1, ""12-3"": 1, ""18-1"": 1, ""18-2"": 1, ""18-4"": 1, ""18-5"": 1, ""19-4"": 1, ""19-2"": 1, ""11-5"": 1, ""11-4"": 1, ""11-1"": 1, ""13-6"": 1, ""11-2"": 1, ""9-5"": 2, ""6-4"": 2, ""12-5"": 1, ""6-2"": 4, ""6-3"": 1, ""6-0"": 1, ""13-3"": 1, ""9-3"": 1, ""5-1"": 1, ""9-4"": 1, ""13-2"": 1, ""7-3"": 1, ""7-2"": 1, ""7-1"": 2, ""9-6"": 2, ""8-4"": 1, ""7-5"": 1, ""17-6"": 1, ""16-0"": 2, ""16-1"": 1, ""16-5"": 1, ""10-0"": 1, ""10-1"": 1, ""8-6"": 1, ""10-3"": 1, ""8-0"": 1, ""10-5"": 3, ""10-6"": 1, ""11-0"": 1}","""checkin"""
1,"""iWPMRDI_MZDZwx0GCJ_imA""","{""19-5"": 1, ""19-1"": 1, ""15-5"": 1, ""13-0"": 1, ""17-3"": 1, ""12-3"": 1}","""checkin"""
2,"""HB8EjFxgesxahG4rvpGzIA""","{""16-4"": 1, ""13-2"": 1, ""17-5"": 1, ""17-4"": 1, ""17-2"": 1, ""13-5"": 1, ""10-5"": 2}","""checkin"""
3,"""VTMqRsk1dXJJaReAQLKIsg""","{""12-5"": 1, ""11-4"": 1, ""17-0"": 1, ""19-1"": 1, ""11-5"": 1, ""17-6"": 1, ""17-3"": 1, ""17-2"": 1, ""13-4"": 2, ""16-5"": 1, ""21-0"": 1, ""22-2"": 1, ""13-2"": 1, ""12-1"": 2}","""checkin"""
4,"""1iPwf-E4e-hA3Td1QonzLQ""","{""13-5"": 7, ""10-5"": 4, ""11-2"": 2, ""15-2"": 1, ""15-5"": 1, ""15-4"": 1, ""10-0"": 1, ""18-1"": 2, ""18-2"": 1, ""18-3"": 2, ""18-5"": 1, ""14-4"": 2, ""14-5"": 2, ""14-6"": 1, ""14-1"": 1, ""14-3"": 1, ""17-1"": 1, ""19-1"": 1, ""19-3"": 1, ""19-2"": 1, ""11-5"": 1, ""11-4"": 1, ""13-1"": 1, ""11-1"": 2, ""11-3"": 1, ""13-4"": 2, ""12-5"": 3, ""12-2"": 3, ""12-3"": 1, ""22-2"": 1, ""9-5"": 1, ""20-1"": 3, ""20-2"": 1, ""8-1"": 1, ""16-3"": 1, ""16-1"": 2, ""17-3"": 2, ""17-2"": 4, ""16-4"": 1, ""8-4"": 2, ""10-1"": 1, ""10-2"": 2, ""21-1"": 1}","""checkin"""


### checkin_info

In [124]:
%%execsql
drop type if exists checkin cascade;
create type checkin as (
    hour integer 
    ,day text
);

create or replace function checkin_pprint(
    checkin text
) returns checkin as 
$$
    import calendar as cal
    val = checkin.split('.')[1].split('-')
    return [int(val[0]), cal.day_name[int(val[1])]]

$$ language plpythonu;

In [132]:
%%execsql
drop table if exists checkins;
create table checkins as
select 
    business_id
    ,(checkin_pprint(key)).*
    ,value::int as num_checkins
from (
select 
    trim(both '"' from business_id) as business_id
    ,(flatten_json(checkin_info, 'checkin')).*
from 
    checkin_raw
) t1
distributed by (business_id);

In [133]:
%%showsql
select * from checkins limit 5

,business_id,hour,day,num_checkins
0,RzhkBV3z_RWw9WHcidKQlw,10,Monday,2
1,RzhkBV3z_RWw9WHcidKQlw,10,Saturday,1
2,RzhkBV3z_RWw9WHcidKQlw,23,Tuesday,3
3,RzhkBV3z_RWw9WHcidKQlw,13,Friday,2
4,RzhkBV3z_RWw9WHcidKQlw,17,Wednesday,1


## Tip Data

In [7]:
%%showsql
select * from yelp.tip_raw limit 5;

,business_id,tip_date,likes,tip_text,type,user_id
0,"""YwUTwzYD7G25DIICSVqFBQ""","""2013-01-11""",0,"""It's in China town and the customer didn't fall in live with these girls so he came back to his room.""","""tip""","""DEgml5fKy8H3GgfI7k0WGA"""
1,"""YwUTwzYD7G25DIICSVqFBQ""","""2014-11-01""",0,"""Video poker, topless dancers...enough said""","""tip""","""qQuMyLtoSwiKNxp57h_sDg"""
2,"""YwUTwzYD7G25DIICSVqFBQ""","""2010-07-09""",0,"""Well, you get what you pay for...""","""tip""","""-m8iD48kKeEF2nmrGGaoUQ"""
3,"""RfR5yc4j9coIPBuxRsOW8Q""","""2010-06-26""",0,"""Again....""","""tip""","""8DdcyZR-THOgH60bIyEnfg"""
4,"""RfR5yc4j9coIPBuxRsOW8Q""","""2010-07-23""",0,"""Get the Adobo Fried Rice Omelette! Won't disappoint! Raimen soup (extra hot) not bad!""","""tip""","""rttCl3o0bLAbFG60Jmv2UA"""


In [134]:
%%execsql
drop table if exists tips;
create table tips as 
select
    trim(both '"' from business_id) as business_id
    ,(trim (both '"' from tip_date))::date as tip_date
    ,likes::integer
    ,trim(both '"' from tip_text) as tip_text
    ,trim(both '"' from user_id) as user_id
from 
    tip_raw
distributed by (business_id);

In [135]:
%%showsql
select * from tips limit 5;

,business_id,tip_date,likes,tip_text,user_id
0,Zx8_4zKdDBSO3qGrkukBIA,2009-11-08,0,12Q FTW!,PNAkqna65eJq8Od5dl1iDA
1,Zx8_4zKdDBSO3qGrkukBIA,2009-12-28,0,Avoid the lunch crowds / lines by coming before 11:30 am.,THC7I7imU3jkhYN1ibSH3w
2,Zx8_4zKdDBSO3qGrkukBIA,2010-01-24,0,Jade red chicken!!,yL9YKQivaqxQClwMxhPajg
3,Zx8_4zKdDBSO3qGrkukBIA,2010-01-28,0,Jade red!,sALB8-F04S9VcZMF_GkGUA
4,Zx8_4zKdDBSO3qGrkukBIA,2010-02-02,0,Suicide plate time #13 9 jfr and black beans. You owe it to yourself to try this,N3L6SDymSRkYQtTiSDmXSg


## Final Parsed Tables

### Business Data
+ businesses
+ business_attributes
+ business_categories
+ business_hours
+ business_neighborhoods


### Review Data
+ review

### User Data
+ users
+ user_elite
+ user_friends

### Checkin Data
+ checkin_info

### Tip Data
+ tip